In [1]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('train.csv')
display(data.head(20))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [11]:
def entropy (col):
    ent = 0.0
    counts = np.unique(col, return_counts = True)
    for x in counts[1]:
        pi = x/ len(col)
        ent += -1*pi*np.log2(pi)
    return ent

In [14]:
def info_gain (df, attr, target):
    tot = entropy(df(target)) 
    val = np.unique(df[attr])
    acc = 0
    for x in val:
        subset = df[df[attr] == x][target]
        x = len(subset) / len(df)
        acc += x * entropy(subset)
    return tot - acc
    
    


### Explicação do Código

- **`tot`**:  
  Calcula a entropia do nosso atributo alvo (`target`).  
  Exemplo: se o target for `"Survived"`, estamos medindo a desordem na coluna `'Survived'`.

- **`val`**:  
  Guarda os valores únicos do atributo escolhido (`attr`).  
  Exemplo: se o atributo for `"Sex"`, `val` será `[male, female]`.

- **Laço `for x in val`:**  
  Para cada valor possível do atributo, por exemplo `x = "male"` ou `x = "female"`:
    - **`subset`**:  
      Filtra o DataFrame, pegando só as linhas onde o atributo (`attr`) é igual a `x`.  
      Depois, seleciona apenas a coluna do target.

    - **`x = len(subset) / len(df)`**:  
      Calcula a proporção de elementos daquele valor em relação ao total.

    - **`acc += x * entropy(subset)`**:  
      Soma a entropia do subset, ponderada pela proporção daquele valor.

- **`return tot - acc`**:  
  O ganho de informação é a diferença entre a entropia total e a entropia condicionada ao atributo.

---

#### Exemplo didático

Se quisermos calcular o ganho de informação ao dividir pelo atributo `"Sex"` para prever `"Survived"`, a função irá:

1. Calcular a entropia de `"Survived"` no DataFrame inteiro.
2. Para cada sexo (`"male"`, `"female"`), calcular a entropia de `"Survived"` apenas nas linhas daquele sexo e ponderar pelo número de ocorrências.
3. Retornar a diferença
    

In [18]:
class DecisionTree:
    def __init__(self, depth = 0, max_depth = 5):
        self.children = {}
        self.attr = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    def train (self, df, features, target):
        if self.depth >= self.max_depth or len(features) == 0 or np.unique(df.target) == 1:
            self.target = df[target].mode()[0] #Pegamos os valores mais comuns
            return
        gains = list(info_gain(df, attr, target) for attr in features)
        best_attr = features[np.argmax(gain)]
        self.attr = best_attr
        self.children = {}

        for x in np.unique(df[best_attr]):
            sub = df[df[best_attr] == x]
            if sub is None:
                self.children = None
            else:
                child = DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
                child.train(sub, list(filter(lambda f: f != best_attr, features)), target)
                self.children[x] = child
    def predict (self, row):
        if self.attr is None or self.children = {}:
            return self.target
        val = row[self.attr]
        if val in self.children and self.children[val] is not None:
            return self.children[val].predict(row)
        else:
            return self.target
                
                
            

In [ ]:
tree = DecisionTree()
tree.train
   
   

## Explicação do código: Classe `DecisionTree`

### 1. Inicialização da Árvore (`__init__`)

Ao criar uma instância da árvore de decisão, o método `__init__` é chamado para inicializar seus atributos:

- **`self.children = {}`**  
  - Um dicionário vazio que irá armazenar os nós filhos da árvore.  
  - Cada chave corresponde a um valor do atributo de decisão (por exemplo, "Sol", "Chuva", etc.), e cada valor é uma subárvore (outro objeto `DecisionTree`).

- **`self.attr`**  
  - Este atributo armazenará o nome do atributo escolhido para dividir os dados naquele nó específico da árvore.  
  - Cada nó da árvore pode ter um `self.attr` diferente, dependendo do melhor atributo selecionado para o split naquele ponto.

- **`self.max_depth` & `self.depth`**  
  - `self.max_depth`: Define a profundidade máxima permitida para a árvore.  
  - `self.depth`: Indica em que nível da árvore o nó atual está (a raiz começa em 0).

- **`self.target`**  
  - Este atributo será preenchido em nós folha com o valor mais comum (modo) da variável alvo (`target`) nos dados daquele nó.

---
## 2.Critérios de Parada na Construção da Árvore de Decisão

Durante o treinamento da árvore de decisão (método `train`), avaliamos algumas condições para determinar se devemos parar a criação de novos nós e transformar o nó atual em uma **folha**. Os principais critérios de parada são:

1. **Limite de Profundidade**  
   - Checamos se `self.depth >= self.max_depth`, ou seja, se a árvore já atingiu o limite máximo de profundidade permitido.

2. **Target Puro**  
   - Verificamos se todos os valores do nosso alvo (`target`) são iguais, como por exemplo `[yes, yes, yes]`. Nesse caso, não faz sentido continuar dividindo, pois todos os exemplos possuem o mesmo rótulo.

3. **Acabaram as Features**  
   - Conferimos se ainda existem atributos (`features`) disponíveis para realizar divisões. Se a lista estiver vazia, não é possível continuar.

---

Quando qualquer uma dessas condições for satisfeita, o nó atual se torna uma folha, e atribuimos a ele o valor **mais comum** do nosso target no subconjunto de dados analisado.  
Por exemplo, se o target no nó é `[yes, yes, yes]`, o valor atribuído será `yes`.

## 3.Escolha da Melhor Feature para Divisão

Na construção da árvore de decisão, seguimos os passos abaixo para selecionar o atributo que mais contribui para a separação dos dados:

1. **Cálculo do Ganho de Informação**  
   - Calculamos o ganho de informação para cada uma das features disponíveis e armazenamos esses valores em uma lista.

2. **Identificação do Maior Ganho**  
   - Usamos `np.argmax(gains)` para encontrar o índice do maior valor de ganho de informação na lista.

3. **Seleção da Melhor Feature**  
   - Selecionamos a feature correspondente a esse índice e armazenamos seu nome na variável `best_attr`.  
   - Esta será a feature utilizada para dividir o nó atual da árvore.
   - 
## 4.Como funciona a recursividade na construção dos filhos da Decision Tree?

No método `train` da nossa árvore de decisão, após escolher o melhor atributo (`best_attr`) para dividir o nó atual, seguimos os seguintes passos para construir seus filhos:

---

1. **Loop sobre os valores únicos do atributo selecionado**  
   Utilizamos um `for` para percorrer cada valor possível de `best_attr`.  
   **Exemplo:**  
   Se `best_attr` for `"sex"` e os valores possíveis forem `[male, female]`, o loop irá funcionar assim:
   - Primeira iteração: `x = "male"`
   - Segunda iteração: `x = "female"`

2. **Filtragem do DataFrame para cada valor**  
   Para cada valor `x`, criamos um subconjunto do DataFrame apenas com as linhas onde `best_attr` é igual a `x`.  
   **Exemplo:**  
   - Para `x = "male"`, `sub` será todas as linhas onde `"sex" == "male"`.
   - Para `x = "female"`, `sub` será todas as linhas onde `"sex" == "female"`.

3. **Criação do nó filho ("Child")**  
   Para cada subconjunto `sub`, instanciamos um novo objeto `DecisionTree`, que será o nó filho correspondente ao valor `x`.  
   **Exemplo:**  
   - O nó `"male"` será um filho do nó atual, responsável por processar apenas os exemplos com `"sex" == "male"`.

4. **Recursividade para treinar o filho**  
   Chamamos `child.train(sub, list(filter(lambda f: f != best_attr, features)), target)`.  
   O que acontece aqui?  
   - Chamamos novamente a função `train`, agora para o filho.  
   - O DataFrame de entrada (`sub`) contém apenas exemplos onde o atributo tem valor `x` (por exemplo, apenas `"male"`).
   - A lista de features é atualizada retirando `best_attr`, pois já foi usada para a divisão.
   - O processo se repete: o filho tentará encontrar o melhor atributo para dividir os dados do seu subconjunto.

   **Exemplo prático:**  
   Imagine que no nó atual, `best_attr` é `"sex"` e estamos na iteração `x = "male"`.  
   - O DataFrame `sub` contém apenas linhas com `"sex" == "male"`.
   - Criamos o nó filho `"male"`.
   - Chamamos `train` para esse filho, que irá procurar agora o melhor atributo (exceto `"sex"`, que já foi usado) para dividir ainda mais o subconjunto de dados.

---

### Resumindo

A cada iteração, dividimos o DataFrame conforme os valores do melhor atributo, criamos um nó filho para cada valor, e chamamos recursivamente o método `train` para cada filho, processando apenas o subconjunto de dados correspondente.  
Dessa forma, a árvore vai “crescendo” de cima para baixo, especializando cada nó conforme os dados vão sendo filtrados!

---

## (Importante) Explicação da função `predict`

A função `predict` é responsável por prever o valor alvo (classe) para uma nova entrada na árvore de decisão. Vamos entender seu funcionamento passo a passo:

---

### 1) Entrada da função

- O parâmetro de entrada é `row`, que representa **uma linha do DataFrame** (ou seja, um exemplo com todos os atributos preenchidos).
- **Exemplo:**  
  ```python
  row = {
      "Sex": "male",
      "Pclass": 3,
      "Embarked": "S",
      ...
  }
  ```

---

### 2) Verificação de nó folha

- O código verifica:
  ```python
  if self.attr is None or self.children == {}:
      return self.target
  ```
- Isso significa que **se o nó atual não tem mais atributo para dividir (`self.attr is None`) ou não tem filhos (`self.children == {}`), então é uma folha**.
- Nesse caso, a função retorna o valor de `self.target`, que é o valor mais comum do target naquele nó.
- **Exemplo:**  
  Se o nó representa todos os passageiros do sexo "male" e a maioria deles não sobreviveu, então `self.target = "Não"`.

---

### 3) Definindo o valor do atributo de decisão

- A linha:
  ```python
  val = row[self.attr]
  ```
- **Significa:**  
  Pegue o valor do atributo que está sendo usado para dividir neste nó.
- **Exemplo:**  
  Se `self.attr == "Sex"` e `row["Sex"] == "male"`, então `val = "male"`.

---

### 4) Decidindo para qual filho ir

- O código verifica:
  ```python
  if val in self.children and self.children[val] is not None:
      return self.children[val].predict(row)
  else:
      return self.target
  ```
- **Ou seja:**
  - Se existe um filho correspondente ao valor de `val` (por exemplo, "male") e esse filho não é nulo, chamamos recursivamente a função `predict` no filho.
  - **Exemplo Prático:**  
    - Se estamos em um nó que divide por `Sex`, e `row["Sex"] == "male"`, então procuramos o filho `self.children["male"]`.
    - Se existir, chamamos `predict` nesse filho, passando a mesma linha.
    - Se esse filho for uma folha (não tem mais atributos para dividir), ele retorna seu `self.target` (por exemplo, "Sim" ou "Não").

- **Se não existir filho para esse valor**, retornamos o valor alvo mais comum do nó atual (`self.target`).

---

### **Fluxo resumido com exemplo**

1. **Começamos na raiz:**  
   - `self.attr == "Sex"`
   - `row["Sex"] == "male"`
   - Vamos para o filho `"male"`

2. **Filho "male":**
   - Suponha que não há mais atributos para dividir (folha)
   - `self.target == "Não"`
   - Retorna `"Não"` como predição

---

### **Resumo**

- A cada chamada, a função verifica se é folha; se não for, escolhe o próximo nó filho de acordo com o valor do atributo de decisão.
- O processo é recursivo e termina em uma folha, retornando a classe prevista.

---